In [ ]:
import requests
import json
API_URL="https://api.spacexdata.com/v5/launches/past"

response = requests.get(API_URL)
data = response.json()

#print(data[0])
print(json.dumps(data[66], indent=2))

{
  "fairings": {
    "reused": false,
    "recovery_attempt": false,
    "recovered": false,
    "ships": []
  },
  "links": {
    "patch": {
      "small": "https://images2.imgbox.com/46/b2/NUQmyHR4_o.png",
      "large": "https://images2.imgbox.com/9e/eb/uGUYOYfZ_o.png"
    },
    "reddit": {
      "campaign": "https://www.reddit.com/r/spacex/comments/91gwfg/merah_putih_telkom4_launch_campaign_thread/",
      "launch": "https://www.reddit.com/r/spacex/comments/9539nr/rspacex_merah_putih_telkom4_official_launch/",
      "media": "https://www.reddit.com/r/spacex/comments/94zr0b/rspacex_merah_putih_media_thread_videos_images/",
      "recovery": null
    },
    "flickr": {
      "small": [],
      "original": [
        "https://farm2.staticflickr.com/1798/43862495212_8fe1688c4b_o.jpg",
        "https://farm1.staticflickr.com/935/43006330655_f1623a3fa1_o.jpg",
        "https://farm1.staticflickr.com/938/28974313177_d16381ff5f_o.jpg",
        "https://farm2.staticflickr.com/1780/43006334

In [ ]:
# after some consideration the columns I am going to extract are next:

cols = [
    "id", "name",
    "date_utc", "date_precision",
    "success", "upcoming", "tbd",
    "rocket",
    "cores[0].core", "cores[0].flight", "cores[0].landing_success",
    "cores[0].landing_type","links.wikipedia",
    "details"
]


In [ ]:
import pandas as pd

df = pd.json_normalize(data)

In [ ]:
df.tail(3)

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,...,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
184,None,NaN,False,NaN,5e9d0d95eda69973a809d1ec,True,[],None,[],[],...,None,https://www.reddit.com/r/spacex/comments/k2ts1...,[],[],None,https://youtu.be/ZlQHF_yBkMQ,ZlQHF_yBkMQ,None,None,NaN
185,None,NaN,False,NaN,5e9d0d95eda69973a809d1ec,True,[],None,[],[],...,None,https://www.reddit.com/r/spacex/comments/k2ts1...,[],[],None,https://youtu.be/VVu2bSJJhgI,VVu2bSJJhgI,None,None,NaN
186,None,NaN,False,NaN,5e9d0d95eda69973a809d1ec,True,[],None,"[{'crew': '62dd7196202306255024d13c', 'role': ...",[],...,None,None,[],[],None,https://youtu.be/5EwW8ZkArL4,5EwW8ZkArL4,None,https://en.wikipedia.org/wiki/SpaceX_Crew-5,NaN


Loads of NaN values appeared, going to explode cores list separately.

In [ ]:
df_exploded = pd.json_normalize(
    data,
    record_path=["cores"],
    meta=[
        "id", "name",
        "date_utc", "date_precision",
        "success", "upcoming", "tbd",
        "rocket",
        ["links", "wikipedia"],
        "details"
    ]
)

print(df_exploded.columns)


Index(['core', 'flight', 'gridfins', 'legs', 'reused', 'landing_attempt',
       'landing_success', 'landing_type', 'landpad', 'id', 'name', 'date_utc',
       'date_precision', 'success', 'upcoming', 'tbd', 'rocket',
       'links.wikipedia', 'details'],
      dtype='object')


In [ ]:
cols_new = ["id", "name",
    "date_utc", "date_precision",
    "success", "upcoming", "tbd",
    "rocket",
    "core", "flight", "landing_success", "landing_type", "links.wikipedia",
    "details"]
df = df_exploded[cols_new]

In [ ]:
df = df.rename(columns={
    "core": "core",
    "flight": "core_flight",
    "landing_success": "core_landing_success",
    "landing_type": "core_landing_type",
    "links.wikipedia": "links_wikipedia"
})


print(df.columns)

Index(['id', 'name', 'date_utc', 'date_precision', 'success', 'upcoming',
       'tbd', 'rocket', 'core', 'core_flight', 'core_landing_success',
       'core_landing_type', 'links_wikipedia', 'details'],
      dtype='object')


In [ ]:
print(df.head(3))

                         id         name                  date_utc  \
0  5eb87cd9ffd86e000604b32a    FalconSat  2006-03-24T22:30:00.000Z   
1  5eb87cdaffd86e000604b32b      DemoSat  2007-03-21T01:10:00.000Z   
2  5eb87cdbffd86e000604b32c  Trailblazer  2008-08-03T03:34:00.000Z   

  date_precision success upcoming    tbd                    rocket  \
0           hour   False    False  False  5e9d0d95eda69955f709d1eb   
1           hour   False    False  False  5e9d0d95eda69955f709d1eb   
2           hour   False    False  False  5e9d0d95eda69955f709d1eb   

                       core  core_flight core_landing_success  \
0  5e9e289df35918033d3b2623            1                 None   
1  5e9e289ef35918416a3b2624            1                 None   
2  5e9e289ef3591814873b2625            1                 None   

  core_landing_type                                    links_wikipedia  \
0              None              https://en.wikipedia.org/wiki/DemoSat   
1              None          

I have concern that I have lost some core records, as it appears as some rockets can be launched with multiple. Let's find out is there any records like this. And decide what to do with them.

In [ ]:
# Count cores per launch
core_counts = [len(item.get("cores", [])) for item in data]

# Detect launches with multiple cores used
multi_core_indices = [i for i, count in enumerate(core_counts) if count > 1]

print("Multi-core launches at :", multi_core_indices)

multi_core_names = [data[i]["name"] for i in multi_core_indices]
print("Multi-core launches:", multi_core_names)

Multi-core launches at : [54, 76, 80]
Multi-core launches: ['Falcon Heavy Test Flight', 'ArabSat 6A', 'STP-2']


In [ ]:
multi_core_data = [data[i] for i in multi_core_indices]
print(json.dumps(multi_core_data, indent=2))

[
  {
    "fairings": {
      "reused": false,
      "recovery_attempt": false,
      "recovered": false,
      "ships": []
    },
    "links": {
      "patch": {
        "small": "https://images2.imgbox.com/cd/48/NVrODg2G_o.png",
        "large": "https://images2.imgbox.com/97/11/mjn87zBs_o.png"
      },
      "reddit": {
        "campaign": "https://www.reddit.com/r/spacex/comments/7hjp03/falcon_heavy_demo_launch_campaign_thread/",
        "launch": "https://www.reddit.com/r/spacex/comments/7vg63x/rspacex_falcon_heavy_test_flight_official_launch/",
        "media": "https://www.reddit.com/r/spacex/comments/7vimtm/rspacex_falcon_heavy_test_flight_media_thread/",
        "recovery": null
      },
      "flickr": {
        "small": [],
        "original": [
          "https://farm5.staticflickr.com/4745/40110304192_b0165b7785_o.jpg",
          "https://farm5.staticflickr.com/4676/40110297852_6173e5cae6_o.jpg",
          "https://farm5.staticflickr.com/4615/40143096241_0324643b5e_o.jpg",

So it seems like there is only 3 launches that used more than one core, so for the simplicity of test task, I am going to remove these records.

In [ ]:
# Count number of cores per launch id
core_counts_pd = df.groupby("id")["core"].count()

In [ ]:
print(core_counts_pd)

id
5eb87cd9ffd86e000604b32a    1
5eb87cdaffd86e000604b32b    1
5eb87cdbffd86e000604b32c    1
5eb87cdbffd86e000604b32d    1
5eb87cdcffd86e000604b32e    1
                           ..
62f3b5290f55c50e192a4e6d    1
62f3b5330f55c50e192a4e6e    1
62f3b53a0f55c50e192a4e6f    1
63161329ffc78f3b8567070b    1
63161339ffc78f3b8567070c    1
Name: core, Length: 187, dtype: int64


In [ ]:
# Keep only single-core launches
single_core_ids = core_counts_pd[core_counts_pd == 1].index
print(single_core_ids)

Index(['5eb87cd9ffd86e000604b32a', '5eb87cdaffd86e000604b32b',
       '5eb87cdbffd86e000604b32c', '5eb87cdbffd86e000604b32d',
       '5eb87cdcffd86e000604b32e', '5eb87cddffd86e000604b32f',
       '5eb87cdeffd86e000604b330', '5eb87cdfffd86e000604b331',
       '5eb87ce0ffd86e000604b332', '5eb87ce1ffd86e000604b333',
       ...
       '62a9f89a20413d2695d8871a', '62a9f8b320413d2695d8871b',
       '62dd70d5202306255024d139', '62f3b4ff0f55c50e192a4e6b',
       '62f3b5200f55c50e192a4e6c', '62f3b5290f55c50e192a4e6d',
       '62f3b5330f55c50e192a4e6e', '62f3b53a0f55c50e192a4e6f',
       '63161329ffc78f3b8567070b', '63161339ffc78f3b8567070c'],
      dtype='object', name='id', length=184)


In [ ]:
df = df[df["id"].isin(single_core_ids)].copy()

Okay, now lets clean the data separated.

In [ ]:
df.head(1)

,id,name,date_utc,date_precision,success,upcoming,tbd,rocket,core,core_flight,core_landing_success,core_landing_type,links_wikipedia,details
0,5eb87cd9ffd86e000604b32a,FalconSat,2006-03-24T22:30:00.000Z,hour,False,False,False,5e9d0d95eda69955f709d1eb,5e9e289df35918033d3b2623,1,None,None,https://en.wikipedia.org/wiki/DemoSat,Engine failure at 33 seconds and loss of vehicle


In [ ]:
df.count()

,0
id,184
name,184
date_utc,184
date_precision,184
success,183
upcoming,184
tbd,184
rocket,184
core,184
core_flight,184


In [ ]:
df.isna().sum()

,0
id,0
name,0
date_utc,0
date_precision,0
success,1
upcoming,0
tbd,0
rocket,0
core,0
core_flight,0


In [ ]:
df["success"].unique()

array([False, True, None], dtype=object)

In [ ]:
df["success"] = df["success"].astype("boolean").fillna(False)

In [ ]:
df["core_landing_success"].unique()

array([None, False, True], dtype=object)

In [ ]:
df[df["core_landing_success"].isna()][["core_landing_success", "details"]]

,core_landing_success,details
0,None,Engine failure at 33 seconds and loss of vehicle
1,None,Successful first stage burn and transition to ...
2,None,Residual stage 1 thrust led to collision betwe...
3,None,Ratsat was carried to orbit on the first succe...
4,None,None
5,None,None
6,None,None
7,None,"Launch was scrubbed on first attempt, second l..."
8,None,"CRS-1 successful, but the secondary payload wa..."
9,None,Last launch of the original Falcon 9 v1.0 laun...


In [ ]:
df["core_landing_success"] = df["core_landing_success"].astype("boolean").fillna(False)

In [ ]:
df["core_landing_type"].unique()

array([None, 'Ocean', 'ASDS', 'RTLS'], dtype=object)

In [ ]:
# I suggest that if core landing type is none then it wasn't designed to land,
# so by researched spacex terminology:
df["core_landing_type"] = df["core_landing_type"].fillna("Expendable")

In [ ]:
df["links_wikipedia"] = df["links_wikipedia"].fillna("No link")
df["details"] = df["details"].fillna("No details available")

In [ ]:
# add some calculated fields
df["launch_year"] = pd.to_datetime(df["date_utc"]).dt.year
df["core_reused"] = df["core_flight"] > 1

In [ ]:
df.head()

,id,name,date_utc,date_precision,success,upcoming,tbd,rocket,core,core_flight,core_landing_success,core_landing_type,links_wikipedia,details,launch_year,core_reused
0,5eb87cd9ffd86e000604b32a,FalconSat,2006-03-24T22:30:00.000Z,hour,False,False,False,5e9d0d95eda69955f709d1eb,5e9e289df35918033d3b2623,1,False,Expendable,https://en.wikipedia.org/wiki/DemoSat,Engine failure at 33 seconds and loss of vehicle,2006,False
1,5eb87cdaffd86e000604b32b,DemoSat,2007-03-21T01:10:00.000Z,hour,False,False,False,5e9d0d95eda69955f709d1eb,5e9e289ef35918416a3b2624,1,False,Expendable,https://en.wikipedia.org/wiki/DemoSat,Successful first stage burn and transition to ...,2007,False
2,5eb87cdbffd86e000604b32c,Trailblazer,2008-08-03T03:34:00.000Z,hour,False,False,False,5e9d0d95eda69955f709d1eb,5e9e289ef3591814873b2625,1,False,Expendable,https://en.wikipedia.org/wiki/Trailblazer_(sat...,Residual stage 1 thrust led to collision betwe...,2008,False
3,5eb87cdbffd86e000604b32d,RatSat,2008-09-28T23:15:00.000Z,hour,True,False,False,5e9d0d95eda69955f709d1eb,5e9e289ef3591855dc3b2626,1,False,Expendable,https://en.wikipedia.org/wiki/Ratsat,Ratsat was carried to orbit on the first succe...,2008,False
4,5eb87cdcffd86e000604b32e,RazakSat,2009-07-13T03:35:00.000Z,hour,True,False,False,5e9d0d95eda69955f709d1eb,5e9e289ef359184f103b2627,1,False,Expendable,https://en.wikipedia.org/wiki/RazakSAT,No details available,2009,False


In [ ]:
df.dtypes

,0
id,object
name,object
date_utc,object
date_precision,object
success,boolean
upcoming,object
tbd,object
rocket,object
core,object
core_flight,int64


In [ ]:
# lets convert all column types
df = df.convert_dtypes()
df["date_utc"] = pd.to_datetime(df["date_utc"], errors="coerce")

In [ ]:
df.dtypes

,0
id,string[python]
name,string[python]
date_utc,"datetime64[ns, UTC]"
date_precision,string[python]
success,boolean
upcoming,boolean
tbd,boolean
rocket,string[python]
core,string[python]
core_flight,Int64


Finally we can convert this into transform script to reuse in etl.